## Import

In [1]:
import pandas as pd
import numpy as np ; np.random.seed(2024)
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

## Read data

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
print('학습데이터 수:', train.shape)
print('평가데이터 수:', test.shape)

학습데이터 수: (252289, 19)
평가데이터 수: (79786, 18)


In [3]:
'''
데이터 설명
- sessionID : 세션 ID
- userID : 사용자 ID
- TARGET : 세션에서 발생한 총 조회수
- browser : 사용된 브라우저
- OS : 사용된 기기의 운영체제
- device : 사용된 기기
- new : 첫 방문 여부 (0: 첫 방문 아님, 1: 첫 방문)
- quality : 세션의 질 (거래 성사를 기준으로 측정된 값, 범위: 1~100)
- duration : 총 세션 시간 (단위: 초)
- bounced : 이탈 여부 (0: 이탈하지 않음, 1: 이탈함)
- transaction : 세션 내에서 발생의 거래의 수
- transaction_revenue : 총 거래 수익
- continent : 세션이 발생한 대륙
- subcontinent : 세션이 발생한 하위 대륙
- country : 세션이 발생한 국가
- traffic_source : 트래픽이 발생한 소스
- traffic_medium : 트래픽 소스의 매체
- keyword : 트래픽 소스의 키워드, 일반적으로 traffic_medium이 organic, cpc인 경우에 설정
- referral_path : traffic_medium이 referral인 경우 설정되는 경로
'''
display(train.head())

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,1.0,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,1.0,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,1.0,Chrome,iOS,mobile,0,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000


## <font color='forestgreen'>Data Cleansing

### 결측치
  - keyword, referral_path에는 traffic_medium값에 따른 결측치가 존재한다.
  - traffic_medium 외 OS, continent, subcontinent, country에도 (not set)으로 결측치가 존재한다.

In [4]:
train.isna().sum()

sessionID                   0
userID                      0
TARGET                      0
browser                     0
OS                          0
device                      0
new                         0
quality                     0
duration                    0
bounced                     0
transaction                 0
transaction_revenue         0
continent                   0
subcontinent                0
country                     0
traffic_source              0
traffic_medium              0
keyword                137675
referral_path          161107
dtype: int64

In [5]:
# keyword, referral_path의 값을 결정하는 traffic_medium의 빈도를 살펴보니 결측치로 판단할 수 있는 (none), (not set)이 있다.
train.traffic_medium.value_counts()

organic      107370
referral      70047
(none)        59022
cpc            9978
affiliate      5365
cpm             501
(not set)         6
Name: traffic_medium, dtype: int64

In [6]:
# (none)은 직접 트래픽이라 트래픽 매체가 없는 것이고 (not set)은 결측치로 봐야한다.
print('(none)의 트래픽 소스:', train.query('traffic_medium == "(none)"').traffic_source.unique())
print('(not set)의 트래픽 소스:', train.query('traffic_medium == "(not set)"').traffic_source.unique())

(none)의 트래픽 소스: ['(direct)']
(not set)의 트래픽 소스: ['Partners' 'google']


In [7]:
# (not set)으로 결측치가 표현된 다른 변수가 있는지 확인한 결과 OS, continent, subcontinent, country에서도 나타난다.
# 대체할 값이 없음으로 그대로 둔다.
train.applymap(lambda x: x=="(not set)").sum()

sessionID                 0
userID                    0
TARGET                    0
browser                   0
OS                     2592
device                    0
new                       0
quality                   0
duration                  0
bounced                   0
transaction               0
transaction_revenue       0
continent               336
subcontinent            336
country                 336
traffic_source            0
traffic_medium            6
keyword                   0
referral_path             0
dtype: int64

In [8]:
# traffic_medium 중 광고로 접속한 cpc, cpm은 ad(광고)로 변경한다.
train['traffic_medium'] = train['traffic_medium'].replace('cpc','ad').replace('cpm','ad')
test['traffic_medium'] = test['traffic_medium'].replace('cpc','ad').replace('cpm','ad')

# (not set)은 결측치로 학습데이터 내 트래픽 소스의 최빈값으로 채운다.
mode = train.groupby('traffic_source')['traffic_medium'].agg(lambda x: x.mode()[0])
ease = train.query('traffic_medium == "(not set)"')['traffic_source'].map(mode)
train.loc[ease.index, 'traffic_medium'] = ease.values

ease = test.query('traffic_medium == "(not set)"')['traffic_source'].map(mode)
test.loc[ease.index, 'traffic_medium'] = ease.values

### 이상치

In [ ]:
# 주요 변수들간 pairplot을 그려본다.
plt.figure(figsize=(20,20))
sns.pairplot(train[['TARGET','new','quality','duration','bounced','transaction','transaction_revenue']], color='forestgreen')
plt.show()

In [ ]:
# 위의 pairplot을 통해 bounced가 1이면 TARGET은 1, duration, transaction, transaction_revenue은 0임을 알아냈다.
# bounced를 Target Encoding하여 Feature Importance가 높게 한다.
train.query('bounced==1')[['TARGET','duration','transaction','transaction_revenue']].drop_duplicates()

In [9]:
# transaction의 최댓값, TARGET의 최댓값은 이상치로 보인다.
outlier = [train.transaction.sort_values().index[-1], train.TARGET.sort_values().index[-1]]
train.drop(outlier, inplace=True)
print('학습데이터 크기:', train.shape)

학습데이터 크기: (252287, 19)


In [10]:
# transaction, transaction_revenue 모두 0의 비율이 커 중요 변수로 작용하지 않을 것이다.
print(train.transaction.value_counts(normalize=True).reset_index().iloc[0])
print(train.transaction_revenue.value_counts(normalize=True).reset_index().iloc[0])

index          0.000000
transaction    0.991252
Name: 0, dtype: float64
index                  0.000000
transaction_revenue    0.991276
Name: 0, dtype: float64


### 변수 정리

In [11]:
# 불필요한 변수는 제거한다.
train.drop(['transaction', 'transaction_revenue','keyword','referral_path'], axis=1, inplace=True)
test.drop(['transaction', 'transaction_revenue','keyword','referral_path'], axis=1, inplace=True)

## <font color='forestgreen'> Feature Engineering

### Feature Generation

In [12]:
train['QD'] = train['quality'] / (train['duration']+1)
test['QD'] = test['quality'] / (test['duration']+1)

### Encoding

In [13]:
# 학습데이터에 한하여 Target encoding 한다.
category = ['device', 'bounced', 'traffic_source', 'traffic_medium']
                        
for i in category:
    rate = train.groupby(i)['TARGET'].mean()
    train[i] = train[i].map(rate)
    test[i] = test[i].map(rate)
    test[i].fillna(train[i].mean(), inplace=True)

### Feature Transformation

In [14]:
numeric = ['quality', 'duration', 'QD']

scaler = MinMaxScaler()
train[numeric] = scaler.fit_transform(train[numeric])
test[numeric] = scaler.transform(test[numeric])

### Feature Selection

In [15]:
# ID열을 제거한다.
train.drop(['sessionID'], axis=1, inplace=True)
test.drop(['sessionID'], axis=1, inplace=True)

In [16]:
unuse = ['continent','userID']

X_train, y_train = train.drop(['TARGET']+unuse, axis=1), train['TARGET'].astype('i')
X_test = test.drop(unuse, axis=1)

In [17]:
tr_x, val_x, tr_y, val_y = train_test_split(X_train, y_train, test_size=0.3, random_state=2024)

model = CatBoostRegressor(iterations=500, cat_features = ['new', 'browser', 'OS', 'subcontinent', 'country'], 
                          objective='Poisson', eval_metric='RMSE',
                          random_state=2024, verbose=False)
model.fit(tr_x, tr_y)
print('RMSE:', mean_squared_error(model.predict(val_x).round(), val_y)**0.5)

RMSE: 2.5541527510018285


In [18]:
# Catboost를 base로 삼고 importance가 낮은 feature는 제거한다.
importance = pd.DataFrame({'feature': model.feature_names_,
                           'importance':model.feature_importances_})
importance

,feature,importance
0,browser,0.370996
1,OS,0.896690
2,device,0.352018
3,new,1.318314
4,quality,17.468105
5,duration,31.645027
6,bounced,16.410333
7,subcontinent,3.873980
8,country,0.195444
9,traffic_source,1.095234


## <font color='forestgreen'> HyperParameter Tunning

## <font color='forestgreen'> Save data

In [19]:
sample = pd.read_csv('data/submission/sample_submission.csv')
sample['TARGET'] = model.predict(X_test).round()

In [20]:
date = str(pd.Timestamp.now())[:16].replace('-','').replace(' ','_').replace(':','')
filename = f'./data/submission/{date}'
sample.to_csv(f'{filename}.csv', index=False)
print(f'{filename} is saved.')

./data/submission/20240222_1739 is saved.
